In [15]:
from datasets import Dataset
from pyprojroot import here

dataset = Dataset.load_from_disk(here('evals/logs/mistral/phaseII/t4d/t4d-/t4d_eval'))
dataset

Dataset({
    features: ['story', 'question', 'answer', 'characters', 'distracting_characters', 'self_discover_input', 'reasoning_structure', 'reasoning', 'trajectory', 'answer_pred'],
    num_rows: 564
})

In [25]:
dataset[16]

{'story': 'Isabella entered the pantry. Emily entered the pantry. Sophia entered the attic. Sophia loves the carrot. The t-shirt is in the envelope. Sophia exited the attic. Isabella exited the pantry. Emily moved the t-shirt to the container. Isabella and Emily plan to use the t-shirt soon.',
 'question': 'Who among the individuals would benefit from receiving helpful information?\nA. Sophia\nB. Isabella\nC. Emily\nD. None of the above',
 'answer': 'Isabella',
 'characters': ['Isabella', 'Emily', 'Sophia'],
 'distracting_characters': ['Sophia'],
 'self_discover_input': 'Observation:\nIsabella entered the pantry. Emily entered the pantry. Sophia entered the attic. Sophia loves the carrot. The t-shirt is in the envelope. Sophia exited the attic. Isabella exited the pantry. Emily moved the t-shirt to the container. Isabella and Emily plan to use the t-shirt soon.\n\nQuestion:\nWho among the individuals would benefit from receiving helpful information?\nA. Sophia\nB. Isabella\nC. Emily\nD

In [29]:
from tqdm import tqdm

t4d = (
    lambda y_i, y_pred_i: y_pred_i
    and y_i in y_pred_i
    and y_i == str(y_pred_i.translate(str.maketrans("", "", "."))[2:])
)

correct = 0
for y_i, y_pred_i in tqdm(zip(dataset["answer"], dataset["answer_pred"])):
    eval_fn = t4d

    if eval_fn(y_i, y_pred_i):
        correct += 1
    else:
        print(y_i, y_pred_i)

print("="*100)
print(correct)
print(correct / dataset.num_rows)

564it [00:00, 188869.26it/s]

Lily B. Carter.
Benjamin B. Aiden.
Ethan B. Elizabeth.
Evelyn A. William.
Mila B. Benjamin.
Jayden A. Charlotte.
Hannah B. Nathan.
Jayden B. Jayden."
Noah B. Jacob.
Ava C. Avery.
Ethan C. Olivia.
Abigail A. Mila.
Amelia B. Lily.
Emily A. Hunter.
Ethan A. Ava.
Owen B. Noah.
Liam D. None of the above.
Jack B. Abigail.
Oliver A. Lily.
545
0.9663120567375887


In [30]:
t4d = (
    lambda y_i, y_pred_i: y_pred_i
    and y_i in y_pred_i
    and y_i == str(y_pred_i.translate(str.maketrans("", "", "."))[2:])
)
bbh = lambda y_i, y_pred_i: y_pred_i and y_i.translate(
    str.maketrans("", "", "()")
) == y_pred_i.translate(str.maketrans("", "", '.()"'))


def calculate_accuracy(benchmark, y: list[str], y_pred: list[str]):
    correct_preds = 0
    for y_i, y_pred_i in tqdm(zip(y, y_pred), desc="Calculating accuracy"):
        if benchmark == "t4d":
            eval_fn = t4d
        elif benchmark == "bbh":
            eval_fn = bbh

        if eval_fn(y_i, y_pred_i):
            correct_preds += 1
    return correct_preds / len(y_pred)

calculate_accuracy("t4d", dataset["answer"], dataset["answer_pred"])

Calculating accuracy: 564it [00:00, 288944.36it/s]


0.9663120567375887

In [17]:
print(dataset[0]["self_discover_input"])

Observation:
Nathan entered the porch. Mia entered the porch. The radish is in the crate. Nathan exited the porch. Mia moved the radish to the drawer. Liam entered the front yard. Nathan and Mia plan to use the radish soon.

Question:
Who among the individuals would benefit from receiving helpful information?
A. Mia
B. Nathan
C. Liam
D. None of the above


In [20]:
print(dataset[0]["reasoning"])

```json
{
    "Identify the core task": {
        "Determine the main question or task that needs to be answered or solved:": "Determine who among the individuals would benefit from receiving helpful information."
    },
    "Understand the context": {
        "Identify the key events and actions described in the observation that are relevant to the task:": "Nathan and Mia enter the porch, Nathan exits, Mia moves the radish, Liam enters the front yard, Nathan and Mia plan to use the radish soon."
    },
    "Break down the observation": {
        "Segment the observation into smaller, sequential, or categorical parts to analyze it more effectively:": [
            "Nathan entered the porch.",
            "Mia entered the porch.",
            "The radish is in the crate.",
            "Nathan exited the porch.",
            "Mia moved the radish to the drawer.",
            "Liam entered the front yard.",
            "Nathan and Mia plan to use the radish soon."
        ]
    },
    "Id